# Verification of Answers to Problems 1

## Foreward Kinematics


Using the values from question 1 part A: The joint angles **$\theta$** are given by 

$$
\theta = \begin{bmatrix} \theta_1 \\ \theta_2 \end{bmatrix} = \begin{bmatrix} 5\pi/6 \\ -3\pi/7 \end{bmatrix}
$$

Using the forward kinematic equations we derived from question 1 we find that the transformation matrix from C to 0 is:

$$
{}^0T_C =
\begin{bmatrix}
-0.5   & -0.1927 & 0.8443 & 1.0754 \\
-0.8660& 0.1113  & -0.4875& 0.3028 \\
0      & -0.9749 & -0.2225& 1.0220 \\
0      & 0       & 0      & 1.0
\end{bmatrix}
$$

The quaternion is: 0.3117 < -0.3909,  0.6771, -0.5400 >

Now, using Mujoco we can plug in these **$\theta$** values and see that the transformation (and quaternion) is the same. 




In [5]:
import mujoco as mj
import mujoco.viewer
import time
import numpy as np
import sympy as sp

# Load the XML model

model = mj.MjModel.from_xml_path("penulum.xml")

data = mj.MjData(model)

joint1_name = "joint1"
joint2_name = "joint2"

joint1_id = model.joint(joint1_name).id
joint2_id = model.joint(joint2_name).id



world_frame_id = 0


frame1_name = "frame1"
frame1_id = model.body(name=frame1_name).id


frame2_name = "frame2"
frame2_id = model.body(name=frame2_name).id

end_effector_body_name="end-effector"
end_effector_body_id = model.body(name=end_effector_body_name).id


theta1_parta = 5 * np.pi / 6
theta2_parta = -3 * np.pi / 7

#set model to the given theta values
data.qpos[joint1_id] = theta1_parta
data.qpos[joint2_id] = theta2_parta

mj.mj_forward(model, data)

def get_spatial_transformation(body_id):
    #Extract the positions
    rot = data.xmat[body_id].reshape(3,3)
    pos = data.xpos[body_id]


    #Convert to transformation matrix
    transformation = np.eye(4) 
    transformation[:3, :3] = rot  
    transformation[:3, 3] = pos 
    return transformation

def quat_print(quat):
    """
    Prints a quaternion in the format: <w, <x, y, z>>
    AI Generated function
    
    Args:
        quat (list or tuple): A quaternion [w, x, y, z]
    """
    if len(quat) != 4:
        raise ValueError("Quaternion must be a list or tuple of 4 elements [w, x, y, z]")
    
    w, x, y, z = [round(val, 4) for val in quat]

    print(f"{w}, <{x}, {y}, {z}>")




#Display
quat_print(data.xquat[end_effector_body_id])
sp.Matrix(np.round(get_spatial_transformation(end_effector_body_id), 4))


0.3117, <-0.3909, 0.6771, -0.54>


Matrix([
[  -0.5, -0.1927,  0.8443, 1.0754],
[-0.866,  0.1113, -0.4875, 0.3028],
[   0.0, -0.9749, -0.2225,  1.022],
[   0.0,     0.0,     0.0,    1.0]])

## Inverse Kinematics

In Question 1d we are asked to find the values of theta corresponding to a given orientation of the camera frame. To verify this calculation we can plug in the values of theta that we found and see if the orientation matches what was given. 


The values of theta we found are:

$$
\left[\begin{matrix}-45.0 & 315.0\\135.0 & 45.0\end{matrix}\right]
$$
 Where each row is one of two possible solutions. Columns 1 and 2 represet **theta1** and **theta2** respectivly.


The expected result is:

$$
k = -\frac{1}{2} x_0 + \frac{1}{2} y_0 - \frac{\sqrt{2}}{2} z_0.
$$

In [6]:
solutions = [(-0.785398163397448, 5.49778714378214),
 (0.785398163397448, 0.785398163397448),
 (2.35619449019234, 0.785398163397448),
 (3.92699081698724, 5.49778714378214)]


theta1_sol1_partd = np.deg2rad(-45)
theta2_sol1_partd = np.deg2rad(135)

theta1_sol2_partd = np.deg2rad(315)
theta2_sol2_partd = np.deg2rad(45)

print("Expected: \n[-0.5         0.5        -0.70710678]\nActual:\n")

for solution in solutions:
    #set model to solution
    data.qpos[joint1_id] = solution[0]
    data.qpos[joint2_id] = solution[1]
    mj.mj_forward(model, data)
    
    k = data.xmat[end_effector_body_id].reshape(3,3)[:,2]
    print(k)

    print()

Expected: 
[-0.5         0.5        -0.70710678]
Actual:

[-0.5         0.5        -0.70710678]

[ 0.5         0.5        -0.70710678]

[-0.5         0.5        -0.70710678]

[ 0.5         0.5        -0.70710678]



As can be seen above, two of the solutions match the given K vector and two do not, which is exactly what was expected. 


# Verification of answers to Problem 2

#### Part A-Foreward kinematics

Using the values from question 1 part A: The joint angles **$\theta$** are given by 

$$
\theta = \begin{bmatrix} \theta_1 \\ \theta_2 \end{bmatrix} = \begin{bmatrix} 5\pi/6 \\ -3\pi/7 \end{bmatrix}
$$

The angular rates of change are also given:

$$
\dot{\boldsymbol{\theta}} = \begin{bmatrix} 1 & 2 \end{bmatrix}^{\top}
$$ 



The goal is to find the spatial and body twist, the spatial twist  found was:

$$
\begin{bmatrix}
-2.07846096908265 \\
1.2 \\
0 \\
1.0 \\
1.73205080756888 \\
1.0
\end{bmatrix}
$$

The body twist was: 

$$
\left[\begin{matrix}-0.779942329745459\\1.77801674716505\\-0.779942329745459\\-2.0\\-0.974927912181824\\-0.222520933956314\end{matrix}\right]
$$


In [ ]:
# --- Reset the simulation and set the configuration ---
mj.mj_resetData(model, data)

# Joint angles (from Question 1 Part A)
theta1_parta = 5 * np.pi / 6
theta2_parta = -3 * np.pi / 7

# Set joint positions
data.qpos[joint1_id] = theta1_parta
data.qpos[joint2_id] = theta2_parta

# Set joint velocities: [1.0, 2.0]
data.qvel[joint1_id] = 1.0
data.qvel[joint2_id] = 2.0

# Compute the forward dynamics to update the simulation state
mj.mj_forward(model, data)

# --- Retrieve Sensor Data to Form the Twists ---

# Get sensor IDs for the new sensor names:
# Spatial sensors (world frame)
spatial_lin_sensor_id = model.sensor("spatial-linear-vel").id # 0
spatial_ang_sensor_id = model.sensor("spatial-ang-vel").id # 1

# Body sensors (end-effector's frame)
body_lin_sensor_id = model.sensor("body-linear-vel").id # 2
body_ang_sensor_id = model.sensor("body-ang-vel").id # 3

# Since each sensor returns a 3D vector, we slice 3 elements starting at each sensor's id.
# Note: sensor data is stored in a 1D array in sensor order.

# Body twist: measured in the end-effector's (body) frame [linear; angular]
body_lin_vel = data.sensordata[6 : 9]
body_ang_vel = data.sensordata[9 : 12]
body_twist = np.concatenate([body_lin_vel, body_ang_vel])
print("\nBody Twist (linear; angular):")
print(body_twist)
# Expected body twist:
# [ -0.779942329745459, 1.77801674716505, -0.779942329745459,
#   -2.0, -0.974927912181824, -0.222520933956314 ]


# find body twist using the spatial twist and an adjoint transformation
from spatialmath.base import tr2adjoint
sp.pprint(sp.simplify(sp.Matrix(tr2adjoint(np.array(get_spatial_transformation(end_effector_body_id)))) @ body_twist))
# Expected spatial twist:
# [ -2.07846096908265, 1.2, 0, 1.0, 1.73205080756888, 1.0 ]



Body Twist (linear; angular):
[-0.77994233  1.77801675 -0.77994233 -2.         -0.97492791 -0.22252093]
[-2.07846096908265  1.2  4.44089209850063e-16  1.0  1.73205080756888  1.0]
